In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import re
import os
import sys
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

/tmp/ipykernel_73121/242889608.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
file_list = os.listdir(path='../data')
file_list_sort = []
for i_i in range(int(len(file_list)/2)):
    for i_j in range(len(file_list)):
        if (file_list[i_j].split('_')[0] == 'answers') and (file_list[i_j]).split('_')[4] == str(i_i) + ".pickle":
            file_list_sort.append(file_list[i_j])
for i_i in range(int(len(file_list)/2)):
    for i_j in range(len(file_list)):
        if (file_list[i_j].split('_')[0] == 'questions') and (file_list[i_j]).split('_')[4] == str(i_i) + ".pickle":
            file_list_sort.append(file_list[i_j])

In [3]:
file_list_sort

['answers_data_python_stackoverflow_0.pickle',
 'answers_data_python_stackoverflow_1.pickle',
 'answers_data_python_stackoverflow_2.pickle',
 'answers_data_python_stackoverflow_3.pickle',
 'answers_data_python_stackoverflow_4.pickle',
 'answers_data_python_stackoverflow_5.pickle',
 'answers_data_python_stackoverflow_6.pickle',
 'answers_data_python_stackoverflow_7.pickle',
 'answers_data_python_stackoverflow_8.pickle',
 'answers_data_python_stackoverflow_9.pickle',
 'answers_data_python_stackoverflow_10.pickle',
 'answers_data_python_stackoverflow_11.pickle',
 'answers_data_python_stackoverflow_12.pickle',
 'answers_data_python_stackoverflow_13.pickle',
 'answers_data_python_stackoverflow_14.pickle',
 'answers_data_python_stackoverflow_15.pickle',
 'answers_data_python_stackoverflow_16.pickle',
 'answers_data_python_stackoverflow_17.pickle',
 'answers_data_python_stackoverflow_18.pickle',
 'answers_data_python_stackoverflow_19.pickle',
 'questions_data_python_stackoverflow_0.pickle',
 

In [4]:
files_200000_questions = []
questions_answers_200000 = []
c = 0
for file_number in range(int(len(file_list_sort)/2)):
    with open(f"../data/{file_list_sort[file_number]}",'rb') as f:
        answer = pickle.load(f)
    with open(f"../data/{file_list_sort[file_number+20]}",'rb') as f:
        question = pickle.load(f)
    [answer.remove(None) for _ in range(len(answer)-8)]
    [question.remove(None) for _ in range(len(question)-8)]
    for T in range(len(answer)):
        for a_q in range(len(answer[T])):
            a_q_new = answer[T][a_q].copy()
            count = 0
            for m_q in range(len(answer[T][a_q])):
                if (len(answer[T][a_q][m_q]) < 20) or (len(re.findall(".+\n",answer[T][a_q][m_q])) == 0):
                    # print(answer[T][a_q][m_q])
                    a_q_new.remove(answer[T][a_q][m_q])
                    count -= 1
                elif answer[T][a_q][m_q][:3] == ">>>":
                    rex_code = re.findall(">>> .+\n|\... .+\n",answer[T][a_q][m_q])
                    a_q_new[m_q+count] = "".join([i.replace("... ","") for i in [i.replace(">>> ","") for i in rex_code]]).strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])
                elif len(re.findall("\n>>>",answer[T][a_q][m_q])) > 0:
                    rex_code = answer[T][a_q][m_q]
                    rex_code = rex_code.replace(">>> ","")
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])
                elif answer[T][a_q][m_q][:1] == "> ":
                    rex_code = answer[T][a_q][m_q].replace("> ","")
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])
                elif answer[T][a_q][m_q][:2] == "-> ":
                    rex_code = answer[T][a_q][m_q].replace("-> ","")
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])

                elif len(re.findall("\n>",answer[T][a_q][m_q])) > 0:
                    rex_code = answer[T][a_q][m_q].replace("> ","")
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])
                elif len(re.findall("\n->",answer[T][a_q][m_q])) > 0:
                    rex_code = answer[T][a_q][m_q].replace("-> ","")
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])

                else:
                    rex_code = answer[T][a_q][m_q]
                    rex_code_ = re.findall("\n<.*>\n|\n\[.*\]\n|\n{.*}\n|\n<.*>|\n\[.*\]|\n{.*}",rex_code)
                    for i in rex_code_:
                        rex_code = rex_code.replace(i[1:],"",1)
                    a_q_new[m_q+count] = rex_code.strip("\n")
                    questions_answers_200000.append([question[T][a_q],a_q_new[m_q+count]])
            answer[T][a_q] = a_q_new

    files_200000_questions.append(answer.copy())
    c += 1
    print("*"*50)
    print(c)
                    
                    

**************************************************
1
**************************************************
2
**************************************************
3
**************************************************
4
**************************************************
5
**************************************************
6
**************************************************
7
**************************************************
8
**************************************************
9
**************************************************
10
**************************************************
11
**************************************************
12
**************************************************
13
**************************************************
14
**************************************************
15
**************************************************
16
**************************************************
17
**************************************************
18
*************************************

In [5]:
len(questions_answers_200000)

585839

In [6]:
questions_answers_200000[17]

['What does the "yield" keyword do in Python?',
 '# return a list                  #  # return a generator\ndef makeRange(n):                #  def makeRange(n):\n    """return [0,1,2,...,n-1]""" #      """return 0,1,2,...,n-1"""\n    TO_RETURN = []               # \n    i = 0                        #      i = 0\n    while i < n:                 #      while i < n:\n        TO_RETURN += [i]         #          yield i\n        i += 1                   #          i += 1\n    return TO_RETURN             # \n\nmakeRange(5)']

In [7]:
with open("question_answer_stackoverflow_python_sort_by_score_top_200000_questions_997579_answers.csv",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["questions","answers"])
    writer.writerows(questions_answers_200000)

In [8]:
questions_answers_200000 = None
answer = None
question = None